In [ ]:
from google.colab import auth, data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Proses autentikasi akun
auth.authenticate_user()
print('Authenticated')

# Buat BigQuery client
project_id = 'dqlab21-480410'
client = bigquery.Client(project = project_id)

Authenticated


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Inisialisasi query yang akan dijalankan
query = """
  select
  l.candidate_id,
  l.position,
  l.experience_years,
  l.major,
  l.applied_date,
  l.kpi_name,
  l.score,

  s.status

from dqlab21-480410.recruitment_analyst.data_long_after_cleaning as l
left join dqlab21-480410.recruitment_analyst.status_candidate as s
on l.candidate_id = s.candidate_id
 """

# Proses ekstraksi data dari BigQuery ke pandas
data_recruitment = client.query(query).to_dataframe()

# Tampilkan hasil
data_recruitment.head(5)

,candidate_id,position,experience_years,major,applied_date,kpi_name,score,status
0,C0003,Recruitment Analyst,0.38,Akuntansi,2024-11-26,kpi_1,3.0,potential
1,C0003,Recruitment Analyst,0.38,Akuntansi,2024-11-26,kpi_2,3.0,potential
2,C0003,Recruitment Analyst,0.38,Akuntansi,2024-11-26,kpi_3,3.0,potential
3,C0003,Recruitment Analyst,0.38,Akuntansi,2024-11-26,kpi_4,4.0,potential
4,C0003,Recruitment Analyst,0.38,Akuntansi,2024-11-26,kpi_5,3.0,potential


**Berikut deskripsi singkat untuk tiap kolom**:

* `candidate_id` : ID unik untuk setiap kandidat. Digunakan sebagai primary key untuk identifikasi kandidat secara individual.
* `position` : Posisi pekerjaan yang dilamar oleh kandidat.  
* `experience_years` : Total pengalaman kerja kandidat dalam satuan tahun (desimal).  
* `major` : Latar belakang pendidikan/jurusan terakhir kandidat.Digunakan untuk menilai kesesuaian pendidikan dengan posisi yang dilamar.
* `applied_date` : Tanggal kandidat mengajukan lamaran kerja.  
* `kpi_name` : Deskripsi nama KPI mulai dari KPI 1-6 dan KPI Major.
* `score` : Nilai perhitungan pada tiap KPInya.
* `status` : Klasifikasi kandidat berdasarkan ranking scoring CVnya top 20% merupakan Strong kandidat, top 50% merupakan kandidat potential dan sisanya weak kandidat.  


## **Data Profiling**

Data profiling adalah proses menganalisis dan menilai kualitas data secara menyeluruh untuk memahami struktur, isi, dan kualitas dataset sebelum digunakan.

Untuk melakukan data profiling pada pandas dapat menggunakan library `ydata-profiling`


In [ ]:
pip install ydata-profiling

In [ ]:
# Import library yang digunakan
from ydata_profiling import ProfileReport

# Lakukan proses profiling data
profile = ProfileReport(
    df = data_recruitment,
    title = 'Data Profiling Report'
)

# Tampilkan hasilnya
display(profile)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 8/8 [00:00<00:00, 50.74it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data_recruitment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2800 entries, 0 to 2799
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   candidate_id      2800 non-null   object 
 1   position          2800 non-null   object 
 2   experience_years  2800 non-null   float64
 3   major             2800 non-null   object 
 4   applied_date      2800 non-null   dbdate 
 5   kpi_name          2800 non-null   object 
 6   score             2800 non-null   float64
 7   status            2800 non-null   object 
dtypes: dbdate(1), float64(2), object(5)
memory usage: 175.1+ KB


In [ ]:
data_recruitment['applied_date']=pd.to_datetime(data_recruitment['applied_date'])

In [ ]:
data_recruitment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2800 entries, 0 to 2799
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   candidate_id      2800 non-null   object        
 1   position          2800 non-null   object        
 2   experience_years  2800 non-null   float64       
 3   major             2800 non-null   object        
 4   applied_date      2800 non-null   datetime64[ns]
 5   kpi_name          2800 non-null   object        
 6   score             2800 non-null   float64       
 7   status            2800 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 175.1+ KB


**1.Total Candidate Apply By Position**

In [ ]:
count_candidate = data_recruitment.groupby('position',as_index= False).agg(candidate_id = ('candidate_id','nunique'))
display(count_candidate)

,position,candidate_id
0,Business Development,132
1,HR Officer,146
2,Recruitment Analyst,122


**2.Average analytics by Position**

In [ ]:
#Aggregasi Data
avg_score = data_recruitment.groupby('position',as_index= False).agg(score = ('score','mean'))
display(avg_score)

,position,score
0,Business Development,3.090909
1,HR Officer,3.043053
2,Recruitment Analyst,3.129977


In [ ]:
avg_score['score'] = avg_score['score']/5*100
display(avg_score)

,position,score
0,Business Development,61.818182
1,HR Officer,60.861057
2,Recruitment Analyst,62.599532


In [ ]:
avg_score['score'] = avg_score['score'].round(2)
display(avg_score)

,position,score
0,Business Development,61.82
1,HR Officer,60.86
2,Recruitment Analyst,62.60


In [ ]:
# Histogram for average Score
import plotly.express as px

fig = px.bar(
    avg_score,
    x='position',
    y='score',
    text='score',
    title='Average Score by Position (Scale 100)',
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.update_traces(
    marker_color='#116466',
    texttemplate='%{text:.2f}',
    textposition='outside'
)

fig.update_layout(
    template='none',
    plot_bgcolor='#FFFDF2',
    paper_bgcolor='#FFFDF2',
    title_x=0.5,
    title_font=dict(size=20),
    font=dict(size=14),
    yaxis=dict(
        title='Average Score (0-100)',
        range=[0, 100],
        gridcolor='rgba(0,0,0,0.1)'
    ),
    xaxis=dict(
        title='Position',
        showgrid=False
    )
)

fig.show()